In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv("./data/the-movies-dataset/ratings_small.csv")# 100k small data set

In [3]:
dataset.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
len(dataset.userId.unique()), len(dataset.movieId.unique())

(671, 9066)

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int64
movieId      100004 non-null int64
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
### trainform data type int to object ( userId, movieId)
dataset.userId = dataset.userId.astype('category').cat.codes.values
dataset.movieId = dataset.movieId.astype('category').cat.codes.values

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
userId       100004 non-null int16
movieId      100004 non-null int16
rating       100004 non-null float64
timestamp    100004 non-null int64
dtypes: float64(1), int16(2), int64(1)
memory usage: 1.9 MB


In [8]:
dataset.head()

,userId,movieId,rating,timestamp
0,0,30,2.5,1260759144
1,0,833,3.0,1260759179
2,0,859,3.0,1260759182
3,0,906,2.0,1260759185
4,0,931,4.0,1260759205


## Model creation

In [9]:
import tensorflow as tf

In [10]:
n_latent_factor_user = 8
n_latent_factor_moive = 10
n_latent_factor_mf = 3
n_users, n_movies = len(dataset.userId.unique()), len(dataset.movieId.unique())

In [11]:
dataset.movieId.unique()

array([  30,  833,  859, ...,  115, 3712, 4629])

In [12]:
# movie_feature_column  = tf.feature_column.categorical_column_with_vocabulary_list(key='terms', vocabulary_list=dataset.movieId.unique())
from tensorflow import keras

In [13]:
movie_input = keras.layers.Input(shape = [1], name = 'Movie')
movie_embedding_mlp = keras.layers.Embedding(n_movies +1 , n_latent_factor_moive, name = 'Movie_embedding_MLP')(movie_input)
movie_vec_mlp = keras.layers.Flatten(name = 'FlattenMovies_MLP')(movie_embedding_mlp)
movie_vec_mlp = keras.layers.Dropout(0.2)(movie_vec_mlp)

movie_embedding_mf = keras.layers.Embedding(n_movies+1 , n_latent_factor_mf, name = 'Movie_embedding_MF')(movie_input)
movie_vec_mf = keras.layers.Flatten(name='FlattenMovies_MF')(movie_embedding_mf)
movie_vec_mf = keras.layers.Dropout(0.2)(movie_vec_mf)

user_input = keras.layers.Input(shape=[1],name='User')
user_embedding_mlp = keras.layers.Embedding(n_users +1 , n_latent_factor_user, name = 'Users_embedding_MLP')(user_input)
user_vec_mlp = keras.layers.Flatten(name = 'FlattenUser_MLP')(user_embedding_mlp)
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_embedding_mf = keras.layers.Embedding(n_users +1 , n_latent_factor_mf, name = 'Users_embedding_MF')(user_input)
user_vec_mf = keras.layers.Flatten(name = 'FlattenUser_MF')(user_embedding_mf)
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)

In [14]:
(movie_embedding_mf, movie_embedding_mlp, user_embedding_mf, user_embedding_mlp)

(<tf.Tensor 'Movie_embedding_MF/GatherV2:0' shape=(?, 1, 3) dtype=float32>,
 <tf.Tensor 'Movie_embedding_MLP/GatherV2:0' shape=(?, 1, 10) dtype=float32>,
 <tf.Tensor 'Users_embedding_MF/GatherV2:0' shape=(?, 1, 3) dtype=float32>,
 <tf.Tensor 'Users_embedding_MLP/GatherV2:0' shape=(?, 1, 8) dtype=float32>)

In [15]:
concat = keras.layers.concatenate([movie_vec_mlp, user_vec_mlp])
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense_1 = keras.layers.Dense(200, name ='FullyConnected_1')(concat_dropout)
dense_batch_1 = keras.layers.BatchNormalization(name='Batch')(dense_1)
dropout_1 = keras.layers.Dropout(0.2, name ='Dropout_1')(dense_batch_1)
dense_2 = keras.layers.Dense(100, name = 'FullyConnected_2')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name = 'Batch_2')(dense_2)

dropout_2 = keras.layers.Dropout(0.2, name='Dropout_2')(dense_batch_2)
dense_3 = keras.layers.Dense(50, name = 'FullyConnected_3')(dropout_2)
dense_4 = keras.layers.Dense(20, name = 'FullyConnected_4', activation='relu')(dense_3)

In [16]:
(movie_vec_mlp.shape , user_vec_mlp.shape)

(TensorShape([Dimension(None), Dimension(10)]),
 TensorShape([Dimension(None), Dimension(8)]))

In [17]:
concat.shape

TensorShape([Dimension(None), Dimension(18)])

In [18]:
(user_vec_mf.shape, movie_vec_mf.shape)

(TensorShape([Dimension(None), Dimension(3)]),
 TensorShape([Dimension(None), Dimension(3)]))

In [19]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [20]:
user_vec_mf

<tf.Tensor 'dropout_3/cond/Merge:0' shape=(?, 3) dtype=float32>

In [21]:
# movie_vec_mf = keras.backend.transpose(movie_vec_mf) # movie_vec_mf shape=(?, 3) to (3,?)

In [22]:
movie_vec_mf

<tf.Tensor 'dropout_1/cond/Merge:0' shape=(?, 3) dtype=float32>

In [23]:
pred_mf = keras.layers.multiply([movie_vec_mf, user_vec_mf], name = 'Elemet_wise_product')
pred_mlp = keras.layers.Dense(1, activation='relu', name = 'Activation_pred')(dense_4)

In [24]:
(pred_mf, pred_mlp)

(<tf.Tensor 'Elemet_wise_product/mul:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'Activation_pred/Relu:0' shape=(?, 1) dtype=float32>)

In [25]:
combine_mlp_mf = keras.layers.concatenate([pred_mf,pred_mlp],name = 'Concat_MF_MLP')
result_combine = keras.layers.Dense(100, name='Combine_MF_MLP')(combine_mlp_mf)

deep_combine = keras.layers.Dense(100, name = 'FullyConnected_5')(result_combine)
result = keras.layers.Dense(1, name='Prediction')(deep_combine)

model = keras.Model([user_input, movie_input], result)
opt = keras.optimizers.Adam(lr = 0.01)
model.compile(optimizer = 'adam', loss = 'mean_absolute_error',metrics=['mae'])

In [26]:
from IPython.display import SVG
# from tensorflow.keras.utils import plot_model

In [27]:
# SVG(plot_model(model, show_shapes=False,to_file='test_model', show_layer_names=True, rankdir='TB')) 
# SVG(plot_model(model, show_shapes=False, show_layer_names=True, rankdir='TB').create(prog='dot',format='svg'))

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie_embedding_MLP (Embedding) (None, 1, 10)        90670       Movie[0][0]                      
__________________________________________________________________________________________________
Users_embedding_MLP (Embedding) (None, 1, 8)         5376        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [30]:
# cv = KFold(n_splits=10, shuffle=True, random_state=0)
# cross_val_score(model, 
#                 dataset.userId, dataset.movieId, scoring="neg_mean_absolute_error", cv=cv)


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
train, test = train_test_split(dataset, test_size=0.2)

In [33]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.00001, patience=10, verbose=1, mode='min') 
### 0.0001 이상의 변화가 없을 때 10회만큼 epoch 더 부여 후 중지

In [34]:
model_epoch_train = model.fit([train.userId, train.movieId], train.rating, epochs=100, verbose=1, 
                          validation_split=0.1, callbacks = [early_stopping])

Train on 72002 samples, validate on 8001 samples
Epoch 1/100
72002/72002 [==============================] - 7s 101us/step - loss: 0.8789 - mean_absolute_error: 0.8789 - val_loss: 0.8333 - val_mean_absolute_error: 0.8333
Epoch 2/100
72002/72002 [==============================] - 7s 92us/step - loss: 0.8244 - mean_absolute_error: 0.8244 - val_loss: 0.7923 - val_mean_absolute_error: 0.7923
Epoch 3/100
72002/72002 [==============================] - 7s 90us/step - loss: 0.7806 - mean_absolute_error: 0.7806 - val_loss: 0.7397 - val_mean_absolute_error: 0.7397
Epoch 4/100
72002/72002 [==============================] - 7s 94us/step - loss: 0.7539 - mean_absolute_error: 0.7539 - val_loss: 0.7324 - val_mean_absolute_error: 0.7324
Epoch 5/100
72002/72002 [==============================] - 7s 94us/step - loss: 0.7368 - mean_absolute_error: 0.7368 - val_loss: 0.7220 - val_mean_absolute_error: 0.7220
Epoch 6/100
72002/72002 [==============================] - 7s 94us/step - loss: 0.7250 - mean_absolu

In [35]:
from sklearn.metrics import mean_absolute_error


In [36]:
y_predict = np.round(model.predict([test.userId, test.movieId]), 0)

In [37]:
print(mean_absolute_error(test.rating, y_predict))
print(mean_absolute_error(test.rating,model.predict([test.userId, test.movieId])))

0.6937153142342883
0.6994522766011172


In [38]:
import h5py

In [39]:
# h5py.run_tests()

In [40]:
tf.keras.models.save_model(model=model, filepath='./data/model/neural_MF_1.h5')